In [1]:
train_dir, dev_dir, test_dir = './supervised/supervised/train.txt', './supervised/supervised/dev.txt', './supervised/supervised/test.txt'

In [2]:
train_sentences = []
raw_lines = []
with open(train_dir, encoding='utf-8') as f:
    raw_lines = f.readlines()
raw_lines[-1] = raw_lines[-1] + '\n'
raw_lines.extend(['\n'])
raw_lines = [line.strip() for line in raw_lines]

beg, end = 0, 0
while end < len(raw_lines):
    if raw_lines[end]:
        end += 1
    else:
        sentence = raw_lines[beg:end]
        train_sentences.append(sentence)
        end += 1
        beg = end
print(len(train_sentences))

valid_sentences = []
raw_lines = []
with open(dev_dir, encoding='utf-8') as f:
    raw_lines = f.readlines()
raw_lines[-1] = raw_lines[-1] + '\n'
raw_lines.extend(['\n'])
raw_lines = [line.strip() for line in raw_lines]

beg, end = 0, 0
while end < len(raw_lines):
    if raw_lines[end]:
        end += 1
    else:
        sentence = raw_lines[beg:end]
        valid_sentences.append(sentence)
        end += 1
        beg = end
print(len(valid_sentences))

test_sentences = []
with open(test_dir, encoding='utf-8') as f:
    lines = f.readlines()
lines[-1] = lines[-1] + '\n'
lines.extend(['\n'])
lines = [line.strip() for line in lines]
beg, end = 0, 0
while end < len(lines):
    if lines[end]:
        end += 1
    else:
        sentence = lines[beg:end]
        test_sentences.append(sentence)
        end += 1
        beg = end
print(len(test_sentences))
print(len(train_sentences) + len(valid_sentences) + len(test_sentences))

131767
18824
37648
188239


In [3]:
washed_train_sentences = []
for sentence in train_sentences:
    for i in range(len(sentence)):
        word_tag = sentence[i]
        tokens = word_tag.split('\t')
        if tokens[1] != 'O':
            if '/' in tokens[1]:
                tokens[1] = tokens[1].replace('/', ',')
                sentence[i] = '\t'.join(tokens)
    O_count = sum([1 if wt.endswith('\tO') else 0 for wt in sentence])
    if O_count != 0 and O_count != len(sentence):
        washed_train_sentences.append(sentence)
print(len(washed_train_sentences))

washed_valid_sentences = []
for sentence in valid_sentences:
    for i in range(len(sentence)):
        word_tag = sentence[i]
        tokens = word_tag.split('\t')
        if tokens[1] != 'O':
            if '/' in tokens[1]:
                tokens[1] = tokens[1].replace('/', ',')
                sentence[i] = '\t'.join(tokens)
    O_count = sum([1 if wt.endswith('\tO') else 0 for wt in sentence])
    if O_count != 0 and O_count != len(sentence):
        washed_valid_sentences.append(sentence)
print(len(washed_valid_sentences))

washed_test_sentences = []
for sentence in test_sentences:
    for i in range(len(sentence)):
        word_tag = sentence[i]
        tokens = word_tag.split('\t')
        if tokens[1] != 'O':
            if '/' in tokens[1]:
                tokens[1] = tokens[1].replace('/', ',')
                sentence[i] = '\t'.join(tokens)
    O_count = sum([1 if wt.endswith('\tO') else 0 for wt in sentence])
    if O_count != 0 and O_count != len(sentence):
        washed_test_sentences.append(sentence)
print(len(washed_test_sentences))
print(len(washed_train_sentences) + len(washed_valid_sentences) + len(washed_test_sentences))

114028
16343
32565
162936


In [4]:
all_sentences = list(set(['\n'.join(s) for s in washed_train_sentences] + ['\n'.join(s) for s in washed_valid_sentences] + ['\n'.join(s) for s in washed_test_sentences]))
print(len(all_sentences))
coarse_classes = set()
for i in range(len(all_sentences)):
    wts = all_sentences[i].split('\n')
    for j in range(len(wts)):
        tokens = wts[j].split('\t')
        if tokens[1] != 'O':
            coarse_tag = tokens[1].split('-')[0]
            wts[j] = tokens[0] + '\t' + coarse_tag
            coarse_classes.add(coarse_tag)
    all_sentences[i] = '\n'.join(wts)
print(f'coarse classes: {coarse_classes}')
coarse_dict = {}

for coarse_class in coarse_classes:
    sentences = []
    for sentence in all_sentences:
        wts = sentence.split('\n')
        for word_tag in wts:
            tag = word_tag.split('\t')[1]
            if tag == coarse_class:
                sentences.append(sentence)
                break
    coarse_dict[coarse_class] = sentences

154387
coarse classes: {'organization', 'building', 'art', 'person', 'product', 'other', 'event', 'location'}


In [5]:
count = 0
tag_count = {}
for cc in coarse_classes:
    tag_count[cc] = len(coarse_dict[cc])
    count += len(coarse_dict[cc])
print(count)
print(tag_count)

261832
{'organization': 55286, 'building': 17189, 'art': 12676, 'person': 55494, 'product': 16208, 'other': 23585, 'event': 15074, 'location': 66320}


In [6]:
import random

train_samples = set()
train_samples_count = {}
for tag in coarse_classes:
    samples = coarse_dict[tag]
    while train_samples_count.get(tag, 0) < 0.7 * tag_count[tag]:
        sample = random.choice(samples)
        if sample not in train_samples:
            satisfied = True
            other_tags = set([wt.split('\t')[1] for wt in sample.split('\n')])
            for other_tag in other_tags:
                if other_tag != 'O' and other_tag != tag:
                    if train_samples_count.get(other_tag, 0) + 1 > 0.7 * tag_count[other_tag]:
                        satisfied = False
                        break
            if satisfied:
                train_samples.add(sample)
                train_samples_count[tag] = train_samples_count.get(tag, 0) + 1
                for other_tag in other_tags:
                    if other_tag != 'O' and other_tag != tag:
                        train_samples_count[other_tag] = train_samples_count.get(other_tag, 0) + 1
print(len(train_samples))
print(train_samples_count)
print(tag_count)

108105
{'organization': 38701, 'product': 11346, 'location': 46424, 'other': 16510, 'event': 10552, 'person': 38846, 'building': 12033, 'art': 8874}
{'organization': 55286, 'building': 17189, 'art': 12676, 'person': 55494, 'product': 16208, 'other': 23585, 'event': 15074, 'location': 66320}


In [8]:
for tag in train_samples_count:
    print(tag, ':', train_samples_count[tag] / tag_count[tag])

organization : 0.7000144702094563
product : 0.7000246791707798
location : 0.7
other : 0.7000211999152003
event : 0.7000132678784662
person : 0.7000036039932245
building : 0.7000407237186573
art : 0.7000631113916062


In [10]:
valid_samples = set()
valid_samples_count = {}
for tag in coarse_classes:
    samples = coarse_dict[tag]
    while valid_samples_count.get(tag, 0) < 0.1 * tag_count[tag]:
        sample = random.choice(samples)
        if sample not in train_samples and sample not in valid_samples:
            satisfied = True
            other_tags = set([wt.split('\t')[1] for wt in sample.split('\n')])
            for other_tag in other_tags:
                if other_tag != 'O' and other_tag != tag:
                    if valid_samples_count.get(other_tag, 0) + 1 > 0.1 * tag_count[other_tag]:
                        satisfied = False
                        break
            if satisfied:
                valid_samples.add(sample)
                valid_samples_count[tag] = valid_samples_count.get(tag, 0) + 1
                for other_tag in other_tags:
                    if other_tag != 'O' and other_tag != tag:
                        valid_samples_count[other_tag] = valid_samples_count.get(other_tag, 0) + 1
print(len(valid_samples))
print(valid_samples_count)
print(tag_count)

15421
{'organization': 5529, 'location': 6632, 'event': 1508, 'person': 5550, 'product': 1621, 'other': 2359, 'art': 1268, 'building': 1719}
{'organization': 55286, 'building': 17189, 'art': 12676, 'person': 55494, 'product': 16208, 'other': 23585, 'event': 15074, 'location': 66320}


In [11]:
for tag in valid_samples_count:
    print(tag, ':', valid_samples_count[tag] / tag_count[tag])

organization : 0.10000723510472814
location : 0.1
event : 0.1000398036353987
person : 0.10001081197967347
product : 0.10001233958538994
other : 0.10002119991520034
art : 0.10003155569580309
building : 0.1000058176740939


In [13]:
test_samples = set()
test_samples_count = {}

for tag in coarse_classes:
    samples = set(coarse_dict[tag]).difference(train_samples.union(valid_samples))
    test_samples_count[tag] = test_samples_count.get(tag, 0) + len(samples)
    for sample in samples:
        test_samples.add(sample)
print(len(test_samples))
print(test_samples_count)
print(tag_count)

30324
{'organization': 10878, 'building': 3397, 'art': 2455, 'person': 10772, 'product': 3203, 'other': 4691, 'event': 2980, 'location': 13039}
{'organization': 55286, 'building': 17189, 'art': 12676, 'person': 55494, 'product': 16208, 'other': 23585, 'event': 15074, 'location': 66320}


In [14]:
for tag in test_samples_count:
    print(tag, ':', test_samples_count[tag] / tag_count[tag])

organization : 0.19675867308179285
building : 0.19762638896968993
art : 0.19367308299147995
person : 0.19411107507117886
product : 0.19761846001974334
other : 0.19889760440958237
event : 0.19769138914687542
location : 0.19660735826296744


In [15]:
with open('./supervised/coarse/train.txt', encoding='utf-8', mode='w') as f:
    f.writelines('\n\n'.join(list(train_samples)))
with open('./supervised/coarse/dev.txt', encoding='utf-8', mode='w') as f:
    f.writelines('\n\n'.join(list(valid_samples)))
with open('./supervised/coarse/test.txt', encoding='utf-8', mode='w') as f:
    f.writelines('\n\n'.join(list(test_samples)))